In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = 30

In [2]:
Filename = "Icon Nov2021.pdf"
period_date = '2021-11-30'
tabname = 'nov2021'

In [3]:
from tabula import read_pdf
df = read_pdf(Filename,pages="all")


In [4]:
def manupulation(df):
    df_flat = df.copy()
    df_flat.rename({'ICON MARI GOLD (BLOCK C)- WELFARE ASSOCIATION' : 'Expense Type', 
                           'Unnamed: 0' : 'Amount'}, axis=1, inplace=True)
    df_flat.dropna(subset=['Expense Type'], inplace=True)
    df_flat['Flat']= df_flat['Expense Type'].apply(lambda x:x if 'Flat no' in x else 'NA' )
    df_flat ['Flat'] = df_flat.loc[(df_flat['Expense Type'].str.contains('Flat no')), 'Flat'].iloc[0]
    df_flat.dropna(inplace=True)
    df_flat['Flat'] = df_flat.loc[(df_flat.Flat.str.contains('Flat no')), 'Flat'].iloc[0]
    df_flat.dropna(inplace=True)
    return df_flat

In [5]:
final_df = pd.DataFrame (columns= ['Expense Type', 'Amount'])
for i in range(25):
    if df[i].columns[0] == 'Watchman Salary':
        df[i] = df[i].columns.to_frame().T.append(df[i], ignore_index=True)
        df[i].columns = range(len(df[i].columns))
        df[i].columns = ['ICON MARI GOLD (BLOCK C)- WELFARE ASSOCIATION', 'Unnamed: 0']
        df[i] = df[i].append(df[i-1],ignore_index=True)
        df_flat = manupulation(df[i])
        
    elif len(df[i].columns) == 1:
        df[i]['Unnamed: 0'] = np.nan
        
    elif len(df[i].columns) == 2:
        df_flat = manupulation(df[i])
    final_df = final_df.append(df_flat, ignore_index=True)

In [6]:
misc_df = final_df[final_df['Expense Type'].isin(['Watchman Salary', 'Common Electricity - to all flats', 'Common Water Expenses(Watchman & Gardening)',
                "Generator Battery,Sanitizer5 lit,Connection charges for Manjeera sump to Normal sump,Water tank cleaning,watch Mobile Reacharge and other Miscellaneous expenditures..",
                "Garden Cutting Hedge,Property Tax link doc Photo copy,External Drainage Cleaning in Aug month,Lift Quarterly Maintenance,Mithilanagar Asso Monthly charges per flat @ Rs 50/- & other Miscellaneous",
                "Diesel Purchase,External Drainage Cleaning,Mithilanagar Asso Monthly charges per flat @ Rs 50/- ,Dussara Mamulu for Garbage,other Miscellaneous",
                "Diesel Purchase,Mithilanagar Asso Monthly charges per flat @ Rs 50/- ,Dussara Mamulu for Line Man,Generator Servicing,Water Pipe 100 ft,Main Gate Lights( Material & Labour) & other Miscellaneous",
                "Drainage Rock Excavation,Laboue Charges for Concrete mix on the pipes, Cement Bags Puchase,Dewali Flower Purchase,Motor Wiring etc..Annexure 01"])]

In [7]:
final_df = final_df[final_df['Expense Type'].isin(['Common Maintenance for each Flat', 'Water tankers bill'
                                     ]) | final_df['Expense Type'].str.contains('Garbage Collection')].drop_duplicates()

In [8]:
final_df['Period Date'] = period_date
misc_df['Period Date'] = period_date

In [9]:
writer = pd.ExcelWriter('icon_marigold.xlsx', engine='openpyxl', mode='a', if_sheet_exists = 'replace')
final_df.to_excel(writer,sheet_name=tabname)
writer.save()
writer.close()

In [10]:
writer = pd.ExcelWriter('icon_marigold_misc.xlsx', engine='openpyxl', mode='a', if_sheet_exists = 'replace')
misc_df.to_excel(writer,sheet_name=tabname)
writer.save()
writer.close()